# Análisis exploratorio

## Carga de los datos

In [205]:
import pandas as pd
from pathlib import Path
import pickle

In [206]:
dir_actual = Path('.')
if (dir_actual / 'dict_dataframes.pkl').exists():
    dataframes = pd.read_pickle('dict_dataframes.pkl')
    print('Leyendo datos de dict_dataframes.pkl')
else:
    directorio = Path('../datos')
    dataframes = {}
    for f in directorio.iterdir():
        if f.suffix == '.xlsx':
            print(f)
            xlsx = pd.ExcelFile(f)
            for sheet in xlsx.sheet_names:
                if sheet != 'WData':
                    df = xlsx.parse(sheet)
                    dataframes[str(f)+"/"+sheet] = df

Leyendo datos de dict_dataframes.pkl


In [207]:
dataframes.keys()

dict_keys(['../datos/Centrífuga 17825.xlsx/Datos', '../datos/Biorreactor 13169.xlsx/Datos', '../datos/Cinéticos IPC.xlsx/Inóculos', '../datos/Cinéticos IPC.xlsx/Cultivos finales', '../datos/Cinéticos IPC.xlsx/Centrifugación', '../datos/Horas inicio fin centrífugas.xlsx/Hoja1', '../datos/Fases producción.xlsx/Preinóculo', '../datos/Fases producción.xlsx/Inóculo', '../datos/Fases producción.xlsx/Cultivo final', '../datos/Centrífuga 14246.xlsx/Datos', '../datos/Biorreactor 14617.xlsx/Datos', '../datos/Biorreactor 13171.xlsx/Datos', '../datos/Biorreactor 14614.xlsx/Datos', '../datos/Biorreactor 13170.xlsx/Datos', '../datos/Biorreactor 14615.xlsx/Datos', '../datos/Biorreactor 14618.xlsx/Datos', '../datos/Movimientos componentes.xlsx/Full1', '../datos/Biorreactor 13172.xlsx/Datos', '../datos/Temperaturas y humedades.xlsx/Datos', '../datos/Biorreactor 14616.xlsx/Datos', '../datos/OF 123456.xlsx/Sheet1', '../datos/Fases producción_test.xlsx/Cultivo final', '../datos/Centrífuga 12912.xlsx/Datos

Tenemos un diccionario con todos los dataframes de los ficheros para poder acceder de manera eficiente.

In [208]:
dataframes['../datos/Fases producción.xlsx/Inóculo'] # ejemplo

,LOTE,ID bioreactor,Fecha/hora inicio,Fecha/hora fin,Volumen de cultivo,Turbidez inicio cultivo,Turbidez final culttivo,Viabilidad final cultivo
0,23019,13171,NaT,NaT,NaN,NaN,32.80,134400000
1,23020,13171,NaT,NaT,NaN,NaN,32.80,134400000
2,23021,14618,NaT,NaT,NaN,NaN,27.84,115200000
3,23022,14618,NaT,NaT,NaN,NaN,27.84,115200000
4,23023,14618,2023-03-27 07:22:00,2023-03-28 07:29:00,155.20,18.24,31.68,106400000
...,...,...,...,...,...,...,...,...
163,24101,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,167.52,16.16,28.08,91200000
164,24103,13171,2024-06-28 07:16:00,2024-06-29 07:06:00,167.52,16.16,28.08,91200000
165,24104,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,169.04,14.40,26.32,82400000
166,24105,13172,2024-07-01 07:01:00,2024-07-02 08:01:00,169.04,14.40,26.32,82400000


Ahora los guardamos en un archivo pickle para luego cargar los datos más rápido en próximas ejecuciones.

In [209]:
if not ((dir_actual / 'dict_dataframes.pkl').exists()):
    with open('dict_dataframes.pkl', 'wb') as f:
        pickle.dump(dataframes, f)

## Creación dataframe de exploración

Vamos a ir creando los dataframes de las variables numéricas de cada uno de los ficheros para luego juntarlos en uno, facilitando su análisis posterior.

In [210]:
# para limpiar los strings de los lotes y utilizar un formato común
def limpiar_string_lote(string: str) -> int:
    if isinstance(string, str):
        if ',' in string:
            string = string.replace(',', '')
        if '/' in string:
            string = string.replace('/', '')
        if 'P' in string:
            string = string.replace('P', '') # hay un lote que tiene una P al principio
        return int(string)
    return string
        

In [217]:
dict_of = {
    'Lote' : list(map(limpiar_string_lote ,dataframes['../datos/OF 123456.xlsx/Sheet1']['Lote'].values)),
    'Cantidad entregada (L)' : dataframes['../datos/OF 123456.xlsx/Sheet1']['Cantidad entregada'].values,
}

df_of = pd.DataFrame(dict_of)
df_of

,Lote,Cantidad entregada (L)
0,23019,13.80
1,23020,13.60
2,23021,13.50
3,23022,13.80
4,23023,13.70
...,...,...
200,24108,13.63
201,24106,13.69
202,24107,13.45
203,24109,11.35


Nos encontramos un problema en la hoja de Preinóculo de Fases de producción.xlsx con el formato de las cabeceras de las columnas, por eso lo corregimos.

In [212]:
new_columns = ['LOTE', 'Fecha/hora inicio', 'Fecha/hora fin', 'pH línea 1',
       'pH línea 2', 'pH línea 3', 'Turbidez línea 1', 'Turbidez línea 2', 'Turbidez línea 3', 'Línea usada línea 1',
       'Línea usada línea 2', 'Línea usada línea 3']

In [213]:
dataframes['../datos/Fases producción.xlsx/Preinóculo'].columns = new_columns
dataframes['../datos/Fases producción.xlsx/Preinóculo'].drop(0, inplace=True)

In [214]:
dict_fases_prod_preinoculo = {
    'Lote' : list(map(limpiar_string_lote , dataframes['../datos/Fases producción.xlsx/Preinóculo']['LOTE'].values)),
    'ph 1' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['pH línea 1'].values,
    'ph 2' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['pH línea 2'].values,
    'ph 3' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['pH línea 3'].values,
    'turbidez 1' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['Turbidez línea 1'].values,
    'turbidez 2' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['Turbidez línea 2'].values,
    'turbidez 3' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['Turbidez línea 3'].values,
    'linea 1' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['Línea usada línea 1'].values,
    'linea 2' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['Línea usada línea 2'].values,
    'linea 3' : dataframes['../datos/Fases producción.xlsx/Preinóculo']['Línea usada línea 3'].values,
}

dict_fases_prod_inoculo = {
    'Lote' : list(map(limpiar_string_lote , dataframes['../datos/Fases producción.xlsx/Inóculo']['LOTE'].values)),
    'Volumen de cultivo' : dataframes['../datos/Fases producción.xlsx/Inóculo']['Volumen de cultivo'].values,
    'Turbidez inicial cultivo inóculo' : dataframes['../datos/Fases producción.xlsx/Inóculo']['Turbidez inicio cultivo'].values,
    'Turbidez final cultivo inóculo' : dataframes['../datos/Fases producción.xlsx/Inóculo']['Turbidez final culttivo'].values,
    'Viabilidad final cultivo' : dataframes['../datos/Fases producción.xlsx/Inóculo']['Viabilidad final cultivo'].values,
}

dict_fases_prod_cultivo_final = {
    'Lote' : list(map(limpiar_string_lote , dataframes['../datos/Fases producción.xlsx/Cultivo final']['LOTE'].values)),
    'Orden en el encadenado' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Orden en el encadenado'].values,
    'Volumen de inóculo utilizado' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Volumen de inóculo utilizado'].values,
    'Turbidez inicio cultivo final' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Turbidez inicio cultivo'].values,
    'Turbidez final cultivo final' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Turbidez fin cultivo'].values,
    'Centrifugación 1 turbidez' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Centrifugación 1 turbidez'].values,
    'Centrigugación 2 turbidez' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Centrifugación 2 turbidez'].values,
    'Producto 1' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Producto 1'].values,
    'Producto 2' : dataframes['../datos/Fases producción.xlsx/Cultivo final']['Producto 2'].values,
}


df_fases_prod_preinoculo = pd.DataFrame(dict_fases_prod_preinoculo)
df_fases_prod_inoculo = pd.DataFrame(dict_fases_prod_inoculo)
df_fases_prod_cultivo_final = pd.DataFrame(dict_fases_prod_cultivo_final)

Ahora vamos a hacer un join de todos los dataframes para tener un único archivo donde vengan recogidas todas las medidas de un lote a lo largo de su producción. No tenemos en cuenta las mediciones en biorreactores ni cinéticas ya que se trata de una análisis exploratorio inicial.

In [215]:
df = pd.merge(df_of, df_fases_prod_preinoculo, on='Lote', how='inner')
df = pd.merge(df, df_fases_prod_inoculo, on='Lote', how='inner')
df = pd.merge(df, df_fases_prod_cultivo_final, on='Lote', how='inner')
df

,Lote,Cantidad entregada (L),ph 1,ph 2,ph 3,turbidez 1,turbidez 2,turbidez 3,linea 1,linea 2,...,Turbidez final cultivo inóculo,Viabilidad final cultivo,Orden en el encadenado,Volumen de inóculo utilizado,Turbidez inicio cultivo final,Turbidez final cultivo final,Centrifugación 1 turbidez,Centrigugación 2 turbidez,Producto 1,Producto 2
0,23023,13.70,5.496,5.504,5.52,28.32,27.92,32,1,1,...,31.68,106400000,1,77.60,17.12,74.40,26.56,20.88,1861.84,2.96
1,23024,13.70,5.496,5.504,5.52,28.32,27.92,32,1,1,...,31.68,106400000,1,76.00,16.56,80.80,24.56,10.40,2161.12,2.80
2,23025,13.70,5.48,5.52,5.52,26.56,27.52,26.4,1,1,...,27.60,84800000,1,77.20,17.76,87.20,30.64,29.36,2044.72,4.48
3,23026,13.80,5.48,5.52,5.52,26.56,27.52,26.4,1,1,...,27.60,84800000,1,78.80,18.24,81.20,26.48,11.60,2263.20,3.44
4,23027,13.70,5.384,5.44,5.4,33.84,30.56,32.48,1,0,...,23.52,104800000,1,83.20,16.88,68.08,26.24,9.84,1407.68,4.08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,24045,13.71,5.512,5.624,5.552,26.08,25.2,28.24,1,1,...,23.20,109600000,1,80.00,17.52,72.48,27.84,17.76,1573.52,5.76
119,24044,13.75,5.424,5.408,5.48,24.4,27.92,24.48,1,1,...,25.28,104800000,1,83.60,19.28,77.52,30.72,20.68,1528.72,5.44
120,24049,13.57,5.424,5.408,5.48,24.4,27.92,24.48,1,1,...,25.28,104800000,1,83.60,18.88,72.64,30.56,17.00,1342.80,4.88
121,24050,13.63,5.496,5.488,5.376,34,27.84,35.12,1,0,...,25.92,102400000,1,84.16,17.76,67.60,29.44,26.64,1422.80,3.68


In [216]:
df.to_csv('dataframe_exploratorio.csv', index=False)